In [1]:
from keras_preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255)

In [2]:
train_generator = datagen.flow_from_directory(
    directory=r"./fruits/fruits-360/Training/",
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=42,
    class_mode="sparse",
    shuffle=True,
    seed=23
)


Found 42798 images belonging to 83 classes.


In [3]:
validate_generator = datagen.flow_from_directory(
    directory=r"./fruits/fruits-360/Test/",
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=285,
    class_mode="sparse",
    shuffle=True,
    seed=23
)

Found 14369 images belonging to 83 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    directory=r"./fruits/fruits-360/test-multiple_fruits/",
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=1,
    class_mode="sparse",
    shuffle=True,
    seed=23
)

In [4]:
import tensorflow as tf

n_inputs = 100
n_channels = 3
n_hidden1 = 1000
n_hidden2 = 400
n_outputs = 83

#create input layer

X = tf.placeholder(tf.float32, shape=(None, n_inputs*n_inputs*n_channels), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [5]:
#Use the tensor flow function layers.dense() to create the deep neural network using the ReLU activation function 
with tf.name_scope("dnn"):
    
    #create first hidden layer
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    
    #create second hidden layer
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    #create output layer
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    

In [6]:
logits.shape

TensorShape([Dimension(None), Dimension(83)])

In [6]:
#Create the cost function - need to penalize the model if it estimates a low probability for the target class
#Use softmax activation and cross entropy on the network before going through the softmax activation function
with tf.name_scope("loss"):
    
    #Use sparse_softmax_cross_entropy_with_logits to handle floating-point rounding errors 
    #(which can cause logit = 0 or 1) when logits are large 
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    #compute the mean cross entropy over all instances
    loss = tf.reduce_mean(xentropy, name="loss")
    

In [7]:
#Next, define a GradientDescentOptimizer that changes the weights to minimize the cost function
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    

In [8]:
#Evaluate the model using accuracy
with tf.name_scope("eval"):
    #correct = tf.nn.in_top_k(tf.reshape(logits, (100*100,83)),y,1)
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#Create node to initialize all variables
init = tf.global_variables_initializer()
#Create saver to save model parameters
saver = tf.train.Saver()

In [9]:
#Defined number of epochs to be run & batch size

n_epochs = 50
batch_size = 42

In [16]:
temp = X_batch.reshape(42,100*100*3)
temp

array([[1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       ...,
       [0.9960785, 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 0.9843138, ..., 1.       , 1.       ,
        1.       ],
       [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
        1.       ]], dtype=float32)

In [10]:
#Train the model
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(42798 // batch_size):
            X_batch, y_batch = next(train_generator)
            sess.run(training_op, feed_dict={X: X_batch.reshape(batch_size,100*100*3), y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch.reshape(batch_size,100*100*3), y: y_batch})
        val_imgs, val_labels = next(validate_generator)
        acc_val = accuracy.eval(feed_dict={X: val_imgs.reshape(285,100*100*3),
                                           y: val_labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.97619045 Val accuracy: 0.79298246
1 Train accuracy: 0.97619045 Val accuracy: 0.7473684
2 Train accuracy: 0.97619045 Val accuracy: 0.9263158
3 Train accuracy: 1.0 Val accuracy: 0.9368421
4 Train accuracy: 0.95238096 Val accuracy: 0.9403509
5 Train accuracy: 1.0 Val accuracy: 0.95438594
6 Train accuracy: 0.97619045 Val accuracy: 0.94385964
7 Train accuracy: 1.0 Val accuracy: 0.9157895
8 Train accuracy: 1.0 Val accuracy: 0.9403509
9 Train accuracy: 1.0 Val accuracy: 0.94736844
10 Train accuracy: 1.0 Val accuracy: 0.94736844
11 Train accuracy: 1.0 Val accuracy: 0.95789474
12 Train accuracy: 1.0 Val accuracy: 0.94736844
13 Train accuracy: 1.0 Val accuracy: 0.94385964
14 Train accuracy: 1.0 Val accuracy: 0.9508772
15 Train accuracy: 1.0 Val accuracy: 0.92280704
16 Train accuracy: 1.0 Val accuracy: 0.9614035
17 Train accuracy: 1.0 Val accuracy: 0.95789474
18 Train accuracy: 1.0 Val accuracy: 0.9614035
19 Train accuracy: 1.0 Val accuracy: 0.9649123
20 Train accuracy: 1.0 Va